In [1]:
import pandas as pd
from datetime import datetime, timedelta
import time
from time import mktime

In [2]:
df_fire = pd.read_csv ('mapdataall_2021.csv',usecols= ['incident_name','incident_date_created', 'incident_county', 'incident_acres_burned'])
df_drought = pd.read_csv ('ca_drought_percent_area_2021.csv',usecols= ['MapDate','County', 'None','D0','D1','D2','D3','D4'])

In [3]:
for index, row in df_fire.iterrows():
    di = datetime.fromtimestamp(mktime(time.strptime(df_fire.loc[index, "incident_date_created"], "%Y-%m-%dT%H:%M:%SZ")))
    d = di.strftime("%Y%m%d")
    incident_county = df_fire.loc[index, "incident_county"]
    
    #print(incident_county)
    #print(d)
    
    drought_rows = df_drought.loc[(df_drought['County'] == incident_county+" County")]
    #print(drought_rows)
    
    for v in drought_rows.itertuples():
        #print(v.MapDate)
        dr_date = datetime.fromtimestamp(mktime(time.strptime(str(v.MapDate), "%Y%m%d")))
        if (di + timedelta(days = 4)) > dr_date:
            #print(v)
            df_fire.loc[index, "None"] = v._3
            df_fire.loc[index, "D0"] = v.D0
            df_fire.loc[index, "D1"] = v.D1
            df_fire.loc[index, "D2"] = v.D2
            df_fire.loc[index, "D3"] = v.D3
            df_fire.loc[index, "D4"] = v.D4
            df_fire.loc[index, "MapDate"] = v.MapDate
            df_fire.loc[index, "County"] = v.County
            break
    
    

In [4]:
print(df_fire.head())

                                   incident_name incident_date_created  \
0                                     Erbes Fire  2021-01-14T16:00:00Z   
1                                 Manzanita Fire  2021-01-19T12:08:00Z   
2  Bonny Doone Complex (Previously Fanning Fire)  2021-01-19T06:48:00Z   
3                              North Butano Fire  2021-01-18T21:00:00Z   
4                               China Grade Fire  2021-01-18T21:00:00Z   

  incident_county  incident_acres_burned  None    D0      D1     D2    D3  \
0         Ventura                  250.0   0.0  1.25   98.75   0.00  0.00   
1          Madera                   23.0   0.0  0.00    0.00  99.74  0.26   
2      Santa Cruz                   20.0   0.0  0.00  100.00   0.00  0.00   
3       San Mateo                   15.0   0.0  0.00   28.29  71.71  0.00   
4      Santa Cruz                   22.0   0.0  0.00  100.00   0.00  0.00   

    D4     MapDate             County  
0  0.0  20210112.0     Ventura County  
1  0.0  2021

In [14]:
df_fire.to_csv('out.csv')

In [15]:
df_fire_final = pd.read_csv ('out.csv')

In [21]:
df_fire_final.head()

,Unnamed: 0,Unnamed: 0.1,incident_name,incident_date_created,incident_county,incident_acres_burned,None,D0,D1,D2,D3,D4,MapDate,County
0,0,0,Erbes Fire,2021-01-14T16:00:00Z,Ventura,250.0,0.0,1.25,98.75,0.00,0.00,0.0,20210112.0,Ventura County
1,1,1,Manzanita Fire,2021-01-19T12:08:00Z,Madera,23.0,0.0,0.00,0.00,99.74,0.26,0.0,20210119.0,Madera County
2,2,2,Bonny Doone Complex (Previously Fanning Fire),2021-01-19T06:48:00Z,Santa Cruz,20.0,0.0,0.00,100.00,0.00,0.00,0.0,20210119.0,Santa Cruz County
3,3,3,North Butano Fire,2021-01-18T21:00:00Z,San Mateo,15.0,0.0,0.00,28.29,71.71,0.00,0.0,20210119.0,San Mateo County
4,4,4,China Grade Fire,2021-01-18T21:00:00Z,Santa Cruz,22.0,0.0,0.00,100.00,0.00,0.00,0.0,20210119.0,Santa Cruz County


In [42]:
bigfire_severe_drought_count = 0
bigfires = df_fire_final.loc[(df_fire_final['incident_acres_burned'] >= 300)]
print("Number of large fires ", bigfires.shape[0])
for v in bigfires.itertuples():
    if (v.D3 + v.D4) > 50:
        bigfire_severe_drought_count = bigfire_severe_drought_count + 1
print("Number of large fires in severe drought conditions ", bigfire_severe_drought_count)

Number of large fires  53
Number of large fires in severe drought conditions  43


In [43]:
smallfire_severe_drought_count = 0
smallfires = df_fire_final.loc[(df_fire_final['incident_acres_burned'] < 300)]
print("Number of small fires ", smallfires.shape[0])
for v in smallfires.itertuples():
    if (v.D3 + v.D4) > 50:
        smallfire_severe_drought_count = smallfire_severe_drought_count + 1
print("Number of small fires in severe drought conditions ", smallfire_severe_drought_count)

Number of small fires  122
Number of small fires in severe drought conditions  85


In [46]:
smallfire_not_severe_drought_count = 0
smallfires = df_fire_final.loc[(df_fire_final['incident_acres_burned'] < 300)]
print("Number of small fires ", smallfires.shape[0])
for v in smallfires.itertuples():
    if (v.D0 + v.D1 + v.D2 ) > 50:
        smallfire_not_severe_drought_count = smallfire_not_severe_drought_count + 1
print("Number of small fires in NOT severe drought conditions ", smallfire_not_severe_drought_count)

Number of small fires  122
Number of small fires in NOT severe drought conditions  37


In [32]:
bigfire_severe_drought


,Unnamed: 0,Unnamed: 0.1,incident_name,incident_date_created,incident_county,incident_acres_burned,None,D0,D1,D2,D3,D4,MapDate,County
5,5,5,Wolf Fire,2021-01-19T11:47:00Z,Kern,685.0,0.0,0.00,37.02,56.54,6.44,0.00,20210119.0,Kern County
11,11,11,Refuge Fire,2021-03-27T17:02:04Z,Siskiyou,873.0,0.0,1.57,16.00,27.15,55.28,0.00,20210330.0,Siskiyou County
20,20,20,North Fire,2021-04-28T23:04:57Z,Los Angeles,650.0,0.0,0.00,3.25,8.55,88.20,0.00,20210427.0,Los Angeles County
24,24,24,Southern Fire,2021-05-02T00:56:04Z,San Diego,5366.0,0.0,86.92,12.98,0.10,0.00,0.00,20210504.0,San Diego County
27,27,27,Gunnison Fire,2021-05-08T13:44:53Z,Butte,549.0,0.0,0.00,0.00,0.00,100.00,0.00,20210511.0,Butte County
31,31,31,Pine Fire,2021-05-12T22:46:39Z,Los Angeles,442.0,0.0,0.00,3.25,8.55,88.20,0.00,20210511.0,Los Angeles County
34,34,34,Palisades Fire,2021-05-14T21:50:41Z,Los Angeles,1202.0,0.0,0.00,3.25,8.55,88.20,0.00,20210518.0,Los Angeles County
35,35,35,Davis Fire,2021-05-18T13:30:00Z,Riverside,300.0,0.0,0.00,0.00,59.42,38.08,2.50,20210518.0,Riverside County
38,38,38,WeeVill Fire,2021-05-20T09:27:00Z,Los Angeles,300.0,0.0,0.00,3.25,8.55,88.20,0.00,20210518.0,Los Angeles County
43,43,43,Mile Fire,2021-05-27T18:44:00Z,Stanislaus,508.0,0.0,0.00,0.00,0.00,99.79,0.21,20210525.0,Stanislaus County


In [7]:
bigfire_rows = df_fire.loc[(df_fire['incident_acres_burned'] >= 300)]

In [10]:
print(len(bigfire_rows))

53
